# Quintessential Agent/Tool

The following code demonstrates how to use Langchain to create and Agent with a Tool and interact with it.

In [8]:
!pip install langchain_google_vertexai langgraph

zsh:1: command not found: pip


In [9]:
import os

from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import StructuredTool
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field

# authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './credentials.json'


Create a simple calculator tool that can multiply two numners.

In [10]:
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Pydantic input model which the Agent will use to slot fill...
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

# build tool from function
calculator = StructuredTool.from_function(
    func=multiply,
    # coroutine=
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=False,
)

Now assemble the agent (llm + tools + prompt + memory)

In [11]:
# instantiate the chat model
model = ChatVertexAI(model='gemini-1.5-pro-002')  # gemini-1.5-flash

# bind the tools
tools = []
tools.append(calculator)
model_with_tools = model.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistent named Amy."),
    ("placeholder", "{messages}"),
    # ("user", "Remember, always be polite!"),
])

# create memory
memory = MemorySaver()

# create agent executor
agent_executor = create_react_agent(model_with_tools, tools, prompt=prompt, checkpointer=memory)

Simple function to perform invoke on the agent executor with the users utternace and return the agents response.

In [12]:
def invoke(utterance: str) -> str:
    thread_id = "abc123"
    response = agent_executor.invoke(
        {
            "messages": [HumanMessage(content=utterance)]
        },
        config={
            "configurable": {"thread_id": thread_id}
        }
    )
    messages = response["messages"]
    ai_message = messages[-1].content
    return ai_message

Chat RPEL

In [13]:
def chat():
    while True:
        utterance = input(">").strip()
        if "q" == utterance.casefold():
            break
        print(invoke(utterance))
    print("Goodbye")

In [15]:
chat()

6 times 7 is 42.

Goodbye
